In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
sub = 'pilot-53'
ses = 'ses-01'
battery = 'camcan'
task = 'finger-tapping'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery)
csv_file = task + '_sub-54_' + ses + '.txt'
tsv_file = task + '_sub-54_' + ses + '.tsv'

In [3]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_file), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [6]:
run_start

4205.8537

In [4]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_file), header=1, sep=' ', skipinitialspace=True)

In [9]:
file_df

,Block#,Trial#,Stimulus#,TrialOnset,StimulusOnset,TrialOffest,KeyPressed#,KeyPressed,ResponseTime,RandOrder
0,0,1,2,4205.85,4205.92,2.2,71.0,g,4206.86,4.0
1,0,2,4,4208.05,4208.06,2.2,188.0,",<",4208.73,3.0
2,0,3,2,4210.26,4210.26,2.2,71.0,g,4210.79,2.0
3,0,4,5,4212.46,4212.46,2.2,71.0,g,4213.34,4.0
4,0,5,2,4214.66,4214.66,2.2,71.0,g,4215.36,3.0
...,...,...,...,...,...,...,...,...,...,...
120,0,116,9,4518.92,4518.92,2.2,0.0,-9,-9.00,4.0
121,0,117,9,4521.12,4521.12,2.2,0.0,-9,-9.00,1.0
122,0,118,9,4523.32,4523.32,2.2,0.0,-9,-9.00,3.0
123,0,119,7,4525.52,4525.52,2.2,89.0,y,4526.69,3.0


In [5]:
# Find rows with trial information
trials = file_df['Block#'] != 'Rest'

In [7]:
# Extract the onsets of stimulus, probe and response
trial_onsets = file_df.loc[trials]['StimulusOnset']

# The stimuli are always on screen for 1s
durations = np.ones(sum(trials))

# The trials are coded numerically, and there are 9 types.
# We are interested in 3 categories:
# “specified” trials, in which subjects are told which finger to press (type 1-4)
# “chosen” trials, in which subjects choose one of many fingers to press (type 5-8)
# null events, where no finger press is required (type 9)
types = file_df.loc[trials]['Stimulus#']
trial_types = np.array(np.zeros(sum(trials), dtype=np.object_))

# Find indices of 'specified' trials and mark as such
s_list = ['1', '2', '3', '4']
for si, s in enumerate(s_list):
    trial_types[types == s] = 'specified'

# Find indices of 'chosen' trials and mark as such
c_list = ['5', '6', '7', '8']
for ci, c in enumerate(c_list):
    trial_types[types == c] = 'chosen'

# Find indices of 'null' trials and mark as such
trial_types[types == '9'] = 'null'

In [92]:
trial_types

array(['Specified', 'Specified', 'Specified', 'Chosen', 'Specified',
       'Null', 'Chosen', 'Specified', 'Chosen', 'Chosen', 'Null', 'Null',
       'Null', 'Chosen', 'Chosen', 'Specified', 'Null', 'Specified',
       'Chosen', 'Null', 'Null', 'Chosen', 'Null', 'Chosen', 'Null',
       'Specified', 'Null', 'Specified', 'Null', 'Specified', 'Chosen',
       'Specified', 'Chosen', 'Chosen', 'Chosen', 'Specified', 'Chosen',
       'Chosen', 'Specified', 'Specified', 'Null', 'Chosen', 'Null',
       'Chosen', 'Specified', 'Null', 'Null', 'Specified', 'Null', 'Null',
       'Null', 'Null', 'Null', 'Chosen', 'Chosen', 'Chosen', 'Chosen',
       'Chosen', 'Specified', 'Null', 'Specified', 'Specified', 'Chosen',
       'Specified', 'Null', 'Chosen', 'Specified', 'Null', 'Specified',
       'Chosen', 'Specified', 'Chosen', 'Chosen', 'Specified', 'Chosen',
       'Chosen', 'Chosen', 'Null', 'Chosen', 'Null', 'Specified',
       'Chosen', 'Specified', 'Specified', 'Chosen', 'Null', 'Specified',


In [9]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)